# RAG

![rag](data/images/rag.jpg "RAG")

In [6]:
# Load OpenAI key from env

import os
from dotenv import load_dotenv
load_dotenv(override=True)

azure_version = "2024-06-01"
azure_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT")
azure_reasoning = os.getenv("AZURE_OPENAI_REASONING")
azure_embeddings = os.getenv("AZURE_OPENAI_EMBEDDINGS")
azure_whisper = os.getenv("AZURE_OPENAI_WHISPER")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_key = os.getenv("AZURE_OPENAI_KEY")

In [7]:
# create llm instance

from langchain_openai.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    api_key=azure_key,
    api_version=azure_version,
    azure_deployment=azure_deployment,
    model=azure_deployment,
    azure_endpoint=azure_endpoint,
)

In [6]:
# invoke llm

llm.invoke("What do you know about Pub Quizzes?")

AIMessage(content='Pub quizzes, also known as trivia nights, are social events typically held in pubs, bars, and restaurants where participants form teams to answer questions on a variety of topics in hopes of winning prizes. The concept originated in the United Kingdom but has become popular in many countries around the world.\n\nHere are some key features of pub quizzes:\n\n1. **Format**: Pub quizzes usually comprise several rounds, each focusing on a different category such as general knowledge, music, sports, history, pop culture, or current events. Some quizzes may have themed rounds or picture rounds.\n\n2. **Team-based**: Participants form teams, often with friends, family, or co-workers. Team sizes can vary, but they typically range from two to six members. The team dynamics add a social aspect to the event.\n\n3. **Question Types**: Questions can be multiple-choice, short answer, true or false, or even involve identifying pictures or songs. Quizmasters—people who host and pres

In [7]:
# invoke llm

answer = llm.invoke("What do you know about Pub Quizzes?")
print(answer.content)

Pub quizzes are a popular form of entertainment found in pubs and bars, predominantly in the United Kingdom but increasingly around the world. These quizzes typically involve patrons forming teams and competing in a series of trivia questions for prizes or bragging rights. Here are some key points about pub quizzes:

### Structure:
1. **Teams**: Participants usually form teams, which can range from just a couple of people to larger groups, depending on the rules set by the pub.
2. **Rounds**: A pub quiz is typically divided into several rounds, each focusing on different topics. Common rounds include general knowledge, current events, music, sports, and themed rounds (e.g., movies, geography).
3. **Question Format**: Questions can be presented in various formats, such as multiple choice, true or false, picture rounds, or open-ended questions.

### Quizmaster:
- The quizmaster is responsible for creating the questions, hosting the event, and ensuring fair play. They might also provide c

In [8]:
# use prompts

from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""You are an encyclopedia.

Question: {input}""")

chain = prompt | llm

answer = chain.invoke({"input": "What are Pub Quizzes?"})
print(answer.content)

Pub quizzes, also known as quiz nights or trivia nights, are informal gatherings where participants engage in trying to answer questions on a wide range of subjects. These events typically take place in pubs, bars, or similar establishments, and they can have both competitive and social components. Participants usually form teams, and a quizmaster or host asks the questions, which can cover topics such as general knowledge, history, sports, entertainment, science, and more.

The structure of pub quizzes can vary, but they often consist of several rounds, each with a different theme or topic. Points are awarded for correct answers, and the team with the most points at the end of the night is declared the winner. Prizes can range from free drinks or meals to cash rewards or other gifts.

Pub quizzes have become a popular social event because they combine elements of fun, teamwork, and intellectual challenge, making them an enjoyable activity for individuals and groups alike.


In [9]:
# reusue prompts

answer = chain.invoke({"input": "What are Pub Quizzes also called?"})
print(answer.content)



Pub quizzes are also commonly called "table quizzes" or "quiz nights."


In [10]:
# stream

for token in chain.stream({"input": "What are Pub Quizzes also called?"}):
    print(token.content)



Pub
 quizzes
 are
 also
 commonly
 known
 as
 "
quiz
 nights
"
 or
 "
tr
ivia
 nights
."
 They
 are
 social
 events
 typically
 held
 in
 pubs
 or
 bars
 where
 teams
 compete
 to
 answer
 general
 knowledge
 questions
,
 often
 across
 a
 variety
 of
 topics
,
 and
 the
 team
 with
 the
 most
 correct
 answers
 wins
 a
 prize
.



In [11]:
# contexts

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

chain = prompt | llm

answer = chain.invoke(
    {
        "input": "What are Pub Quizzes also called?",
        "context": "A pub quiz is a quiz held in a pub or bar. These events are also called quiz nights,[1] trivia nights,[2] or bar trivia[3] and may be held in other settings. The pub quiz is a modern example of a pub game, and often attempts to lure customers to the establishment on quieter days. The pub quiz has become part of British culture since its popularization in the UK in the 1970s by Burns and Porter, although the first mentions in print can be traced to 1959.[4][5] It then became a staple in Irish pub culture, and its popularity has continued to spread internationally. Although different pub quizzes can cover a range of formats and topics, they have many features in common. Most quizzes have a limited number of team members, offer prizes for winning teams, and distinguish rounds by category or theme.",
    }
)
print(answer.content)

Pub quizzes are also called quiz nights, trivia nights, or bar trivia.


In [12]:
# documents

from langchain_core.documents import Document
from langchain.chains.combine_documents import create_stuff_documents_chain


documents = [
    Document(
        page_content="A pub quiz is a quiz held in a pub or bar. These events are also called quiz nights,[1] trivia nights,[2] or bar trivia[3] and may be held in other settings. The pub quiz is a modern example of a pub game, and often attempts to lure customers to the establishment on quieter days. The pub quiz has become part of British culture since its popularization in the UK in the 1970s by Burns and Porter, although the first mentions in print can be traced to 1959.[4][5] It then became a staple in Irish pub culture, and its popularity has continued to spread internationally. Although different pub quizzes can cover a range of formats and topics, they have many features in common. Most quizzes have a limited number of team members, offer prizes for winning teams, and distinguish rounds by category or theme.",
        metadata={
            "source": "wikipedia"
        }
    )
]

document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
)

answer = document_chain.invoke(
    {
        "input": "What are Pub Quizzes also called?",
        "context": documents,
    }
)
print(answer)

Pub quizzes are also called quiz nights, trivia nights, or bar trivia.


In [13]:
answer = document_chain.invoke(
    {
        "input": "What are Pub Quizzes also called and what is the source?",
        "context": documents,
    }
)
print(answer)

Pub quizzes are also called quiz nights, trivia nights, or bar trivia. This information is sourced from the provided context.


In [14]:
# document prompts

document_prompt = ChatPromptTemplate.from_template("""Content: {page_content}                             
Source: {source}""")

document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt,
    document_prompt=document_prompt,
)

answer = document_chain.invoke(
    {
        "input": "What are Pub Quizzes also called and what is the source?",
        "context": documents,
    }
)
print(answer)

Pub quizzes are also called quiz nights, trivia nights, or bar trivia. The source is Wikipedia.


In [15]:
import wikipedia

wikipedia.search("Pub Quiz")

['Pub quiz',
 "Jay's Virtual Pub Quiz",
 'The Big Fat Quiz of the Year',
 'Pub',
 "Al Murray's Great British Pub Quiz",
 'Quiz',
 'Sporcle',
 'Mark Labbett',
 'Paul Sinha',
 'Al Murray']

In [16]:
wikipedia.page("Pub Quiz").content

'A pub quiz is a quiz held in a pub or bar. These events are also called quiz nights, trivia nights, or bar trivia and may be held in other settings. The pub quiz is a modern example of a pub game, and often attempts to lure customers to the establishment on quieter days. The pub quiz has become part of British culture since its popularization in the UK in the 1970s by Burns and Porter, although the first mentions in print can be traced to 1959. It then became a staple in Irish pub culture, and its popularity has continued to spread internationally. Although different pub quizzes can cover a range of formats and topics, they have many features in common. Most quizzes have a limited number of team members, offer prizes for winning teams, and distinguish rounds by category or theme.\n\n\n== History ==\nThe origins of the pub quiz are relatively unknown. \nIn 1946, a night in Yorkshire is mentioned in the Guinness Book of Records.\nIn the late-1950s in Merseyside and Lancashire, "about 4,

In [17]:
from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.runnables import RunnableLambda

def retrieve_wikipedia_page(input):
    search = wikipedia.search(input["input"])
    if not search:
        return []
    page = wikipedia.page(search[0])
    return [Document(
        page_content=page.content,
        metadata={
            "source": "wikipedia"
        }
    )]

runnable = RunnableLambda(retrieve_wikipedia_page)
runnable.invoke({"input": "Pub Quiz"})

chain = create_retrieval_chain(
    runnable, document_chain
)
chain.invoke(
    {
        "input": "What are Pub Quizzes also called and what is the source?",
    }
)

{'input': 'What are Pub Quizzes also called and what is the source?',
 'context': [Document(metadata={'source': 'wikipedia'}, page_content='A pub quiz is a quiz held in a pub or bar. These events are also called quiz nights, trivia nights, or bar trivia and may be held in other settings. The pub quiz is a modern example of a pub game, and often attempts to lure customers to the establishment on quieter days. The pub quiz has become part of British culture since its popularization in the UK in the 1970s by Burns and Porter, although the first mentions in print can be traced to 1959. It then became a staple in Irish pub culture, and its popularity has continued to spread internationally. Although different pub quizzes can cover a range of formats and topics, they have many features in common. Most quizzes have a limited number of team members, offer prizes for winning teams, and distinguish rounds by category or theme.\n\n\n== History ==\nThe origins of the pub quiz are relatively unknow

In [18]:
chain = create_retrieval_chain(
    runnable, document_chain
)
chain.invoke(
    {
        "input": "What are Pub Quizzes also called and what is the source?",
    }
)

{'input': 'What are Pub Quizzes also called and what is the source?',
 'context': [Document(metadata={'source': 'wikipedia'}, page_content='A pub quiz is a quiz held in a pub or bar. These events are also called quiz nights, trivia nights, or bar trivia and may be held in other settings. The pub quiz is a modern example of a pub game, and often attempts to lure customers to the establishment on quieter days. The pub quiz has become part of British culture since its popularization in the UK in the 1970s by Burns and Porter, although the first mentions in print can be traced to 1959. It then became a staple in Irish pub culture, and its popularity has continued to spread internationally. Although different pub quizzes can cover a range of formats and topics, they have many features in common. Most quizzes have a limited number of team members, offer prizes for winning teams, and distinguish rounds by category or theme.\n\n\n== History ==\nThe origins of the pub quiz are relatively unknow

# FAISS

In [ ]:
from uuid import uuid4
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import AzureOpenAIEmbeddings

# load data
loader = TextLoader(r"./PubTexts/guides/userguide2016.txt", encoding="utf-8")
data = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100, separators=[".", "\n"])
documents = splitter.split_documents(data)

embeddings = AzureOpenAIEmbeddings(
    api_key=azure_key,
    api_version=azure_version,
    azure_deployment=azure_embeddings,
    azure_endpoint=azure_endpoint,
)

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['0e8dcdee-1826-4ac7-a116-c1139ec3daf1',
 '94df4c71-9e6a-46ce-b0ba-1299ae2c47e4',
 '7ab6f74e-ee02-4b6d-8be9-8b2ea2d17ba7']

In [20]:

from langchain.chains.retrieval import create_retrieval_chain

retriever = vector_store.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

retriever.invoke("Pub quiz")

[Document(id='0e8dcdee-1826-4ac7-a116-c1139ec3daf1', metadata={'source': './PubTexts/guides/userguide2016.txt'}, page_content='Tech Innovators Smart Home System User Guide\nTable of Contents\nIntroduction\nProduct Overview\nGetting Started\nDevice Installation\nSmart Hub\nSmart Lights\nSmart Thermostat\nSmart Security Camera\nSmart Door Lock\nConnecting Devices to the Smart Hub\nUsing the Mobile App\nApp Installation\nDevice Management\nCreating Automation Rules\nTroubleshooting\nMaintenance and Care\nWarranty and Support\nSafety Information\nContact Information\n1. Introduction\nWelcome to the Tech Innovators Smart Home System! This user guide will help you set up and use your new smart home devices to create a more convenient, secure, and energy-efficient living environment.\n\n2. Product Overview\nThe Tech Innovators Smart Home System includes the following devices:\n\nSmart Hub: The central control unit for all your smart devices.\nSmart Lights: Energy-efficient LED bulbs that can 

## Vector Stores

In [11]:
# vector stores

from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from langchain_chroma import Chroma


# load data
loader = TextLoader(r"./PubTexts/internal/budget.txt", encoding="utf-8")
data = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100, separators=[".", "\n"])
documents = splitter.split_documents(data)

embeddings = AzureOpenAIEmbeddings(
    api_key=azure_key,
    api_version=azure_version,
    azure_deployment=azure_embeddings,
    azure_endpoint=azure_endpoint,
)
db = Chroma.from_documents(documents, embeddings, persist_directory="./PubDatabase/chroma")

for document in documents:
    print(document)
    print("------------------------")

page_content='Tech Innovators Inc. Team Event Budget Application Guide

1. Introduction Organizing team events is a great way to boost morale and foster collaboration. This guide will help you navigate the process of applying for a budget for your team event.

2. Plan Your Event

Define the Purpose: Clearly outline the purpose of the event (e.g., team building, training, celebration).
Set Objectives: Identify the goals you aim to achieve with the event.
Draft a Proposal: Prepare a detailed proposal including the event’s purpose, objectives, and expected outcomes.
3. Estimate the Budget

List Expenses: Itemize all potential expenses, such as venue rental, catering, transportation, materials, and any other costs.
Get Quotes: Obtain quotes from vendors to ensure accurate budgeting.
Create a Budget Plan: Summarize the total estimated cost and break it down into categories.
4. Submit a Budget Request

Step 1: Access the HR Portal
Log in to the Tech Innovators HR portal using your employee c

In [ ]:
# retriever

from langchain.chains.retrieval import create_retrieval_chain

db = Chroma(persist_directory="./chroma_1/presentation", embedding_function=embeddings)
retriever = db.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

retriever.invoke("Pub quiz")

In [ ]:
retrieval_chain.invoke({"input": "LLM"})

In [ ]:
answer = retrieval_chain.invoke({"input": "What are Pub Quizzes also called? Cite the Source!"})
print(answer["answer"])

In [ ]:
answer = retrieval_chain.invoke({"input": "Give a concise description of a LLM. Cite the Source! Write every sentence in a newline."})
print(answer["answer"])

## Summary

In [ ]:
summary_prompt = ChatPromptTemplate.from_template("""Please summarize the following piece of text.
Respond in a manner that a 5 year old would understand.

Text: {context}""")

summary_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=summary_prompt,
    document_prompt=document_prompt,
)

answer = summary_chain.invoke(
    {"context": documents}
)
print(answer)

# Map Reduce

In [ ]:
from functools import partial

from langchain.chains.combine_documents import collapse_docs, split_list_of_docs
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.prompts import format_document
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

partial_format_document = partial(format_document, prompt=document_prompt)

# The chain we'll apply to each individual document.
# Returns a summary of the document.
map_chain = (
    {"context": partial_format_document}
    | summary_prompt
    | llm
    | StrOutputParser()
)

# A wrapper chain to keep the original Document metadata
map_as_doc_chain = (
    RunnableParallel({"doc": RunnablePassthrough(), "content": map_chain})
    | (lambda x: Document(page_content=x["content"], metadata=x["doc"].metadata))
).with_config(run_name="Summarize (return doc)")

# The chain we'll repeatedly apply to collapse subsets of the documents
# into a consolidate document until the total token size of our
# documents is below some max size.
def format_docs(docs):
    return "\n\n".join(partial_format_document(doc) for doc in docs)

collapse_chain = (
    {"context": format_docs}
    | PromptTemplate.from_template("Collapse this content:\n\n{context}")
    | llm
    | StrOutputParser()
)

def get_num_tokens(docs):
    return llm.get_num_tokens(format_docs(docs))

def collapse(
    docs,
    config,
    token_max=400,
):
    collapse_ct = 1
    while get_num_tokens(docs) > token_max:
        config["run_name"] = f"Collapse {collapse_ct}"
        invoke = partial(collapse_chain.invoke, config=config)
        split_docs = split_list_of_docs(docs, get_num_tokens, token_max)
        docs = [collapse_docs(_docs, invoke) for _docs in split_docs]
        collapse_ct += 1
    return docs

# The chain we'll use to combine our individual document summaries
# (or summaries over subset of documents if we had to collapse the map results)
# into a final summary.

reduce_chain = (
    {"context": format_docs}
    | PromptTemplate.from_template("Combine these summaries:\n\n{context}")
    | llm
    | StrOutputParser()
).with_config(run_name="Reduce")

# The final full chain
map_reduce = (map_as_doc_chain.map() | collapse | reduce_chain).with_config(run_name="Map reduce")

answer = map_reduce.invoke(
    input=documents[:5],
    config={"max_concurrency": 5},
)
print(answer)

NameError: name 'document_prompt' is not defined

# Reasoning

In [ ]:
reason_prompt = ChatPromptTemplate.from_template("""You are a teacher answering questions of students. Provide thorough reasoning for your answer.
Question: {input}""")

reason_chain = reason_prompt | llm | StrOutputParser()

answer = reason_chain.invoke({"input": "Ein Sack mit 4 Kilo Kartoffeln kostet 8 Euro. Wie teuer sind 3 Kilogram der Kartoffeln?"})
print(answer)

Um den Preis für 3 Kilogramm Kartoffeln zu berechnen, müssen wir zuerst den Preis pro Kilogramm ermitteln.

Ein Sack mit 4 Kilogramm Kartoffeln kostet 8 Euro. Daher beträgt der Preis pro Kilogramm Kartoffeln:
\[ \text{Preis pro Kilogramm} = \frac{8 \text{ Euro}}{4 \text{ Kilogramm}} = 2 \text{ Euro pro Kilogramm} \]

Nun wissen wir, dass 1 Kilogramm Kartoffeln 2 Euro kostet. Um den Preis für 3 Kilogramm zu berechnen, multiplizieren wir den Preis pro Kilogramm mit der Anzahl der Kilogramm, die wir kaufen möchten:
\[ \text{Preis für 3 Kilogramm} = 3 \text{ Kilogramm} \times 2 \text{ Euro pro Kilogramm} = 6 \text{ Euro} \]

Daher kosten 3 Kilogramm Kartoffeln 6 Euro.


In [ ]:
answer = reason_chain.invoke({"input": """
In Deutschland verbraucht jeder Mensch im Jahr durchschnittlich 350 kg Holz (Papier, Brennholz, Möbel usw.). Ein Festmeter Holz (ein m³ vollgefüllt) wiegt etwa 700 kg.
Ein hundertjähriger Baum hat ca. 2,5 Festmeter. Ein Menschenleben dauert etwa 75 Jahre.

    Wie viel wiegt ein solcher Baum?
    In welcher Zeit verbraucht man das Holz eines 100-jährigen Baumes?
    Wie viel solche Bäume verbrauchen wir im Leben?
    Wie viele „Baumjahre" verbrauchen wir im Leben?
"""})  # Ergebnis: 1.) 1750 kg      2.) 5 Jahren      3.) 15 Bäume       4.) 1500 Baumjahre
print(answer)

: 

In [ ]:
llm_reason = AzureChatOpenAI(
    api_key=azure_key,
    api_version=azure_version,
    azure_deployment=azure_reasoning,
    model=azure_deployment,
    azure_endpoint=azure_endpoint,
)

reason_chain = reason_prompt | llm_reason | StrOutputParser()

answer = reason_chain.invoke({"input": """
Leonardo macht sich wieder auf den Weg mit seinem Bugatti Veyron.
Er fährt wieder von Pisa nach Mailand mit durchschnittlich 380 km/h.
Er braucht dafür eine Dreiviertelstunde. Auf dem Rückweg landet er wiederum in einem Stau. Der Stau ist 10 km lang und er braucht dafür 1,5 Stunden.
Danach kommt nochmals 60 km langsam fahrender Verkehr von 40 km/h. Danach kann Leonardo wieder normal weiterfahren.
Insgesamt ist seine Durchschnittsgeschwindigkeit, gemessen für den Hin- und Rückweg, 114 km/h.
Was ist seine Durchschnittsgeschwindigkeit auf dem Rückweg, auf dem Abschnitt ohne Stau oder langsam fahrenden Verkehr?"""})
print(answer)

: 

## Few shotting

In [ ]:
intent_prompt = ChatPromptTemplate.from_template(
"""A question can have an intent of either Booking a flight, Getting a reservation or ordering food.
Now tell me which intent the following question has.
                                            
Question: {input}""")

intent_chain = intent_prompt | llm
answer = intent_chain.invoke({"input": "Let's go have some food at Wendy's."})
print(answer.content)

: 

In [ ]:
intent_prompt = ChatPromptTemplate.from_template(
"""A question can have an intent of either Booking a flight, Getting a reservation or ordering food.
Now tell me which intent the following question has. Only answer with two/three words.
                                            
Question: {input}""")

intent_chain = intent_prompt | llm
answer = intent_chain.invoke({"input": "Let's go have some food at Wendy's."})
print(answer.content)

: 

In [ ]:
intent_prompt = ChatPromptTemplate.from_template(
"""A question can have an intent of either Booking a flight, Getting a reservation or ordering food.

Consider the following examples:

Question: Let's go to Tokyo next week.
Answer: Booking a flight

Question: Get some pizza delivered for this evening.
Answer: Ordering food

Question: How about a movie at 8pm at the cinema?
Anmswer: Getting a reservation

Now tell me which intent the follolwing question has. Only answer with two/three words.
Question: {input}""")

intent_chain = intent_prompt | llm
answer = intent_chain.invoke({"input": "Let's go have some food at Wendy's."})
print(answer.content)

: 

## Agent and Tools

In [ ]:
answer = llm.invoke("What did Frank-Walter Steinmeier say in his christmas speech 2023?")
print(answer.content)

: 

In [ ]:
from langchain.tools import Tool
from langchain.tools.ddg_search import DuckDuckGoSearchRun

ddg = DuckDuckGoSearchRun()

ddg_tool = Tool.from_function(
    func = ddg.run,
    name = "duck_duck_go_search",
    description = "Search DuckDuckGo for a query about current events.",
)

tools = [ddg_tool]

ddg_tool.run("What did Frank-Walter Steinmeier say in his christmas speech 2023?")

: 

In [ ]:
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import create_react_agent

agent_prompt = SystemMessage(content="You are a helpful assistant. You can use the tools provided to help answer questions.")
agent = create_react_agent(
    tools=tools, model=llm, prompt=agent_prompt, debug=True,
)
agent.invoke({"messages": "What did Frank-Walter Steinmeier say in his christmas speech 2023?"})

: 

In [ ]:
from IPython.display import Image, display

display(Image(agent.get_graph().draw_mermaid_png()))

: 

In [ ]:
new_agent_prompt = SystemMessage(content="You are a great AI-Assistant and only answer in 10 words. You can use the tools provided to help answer questions.")
agent = create_react_agent(
    tools=tools, model=llm, prompt=new_agent_prompt, debug=True
)
agent.invoke({"messages": "What did Frank-Walter Steinmeier say in his christmas speech 2023?"})

: 

In [ ]:
result = agent.invoke({"messages": "What did Frank-Walter Steinmeier say in his christmas speech 2023?"})
result["messages"][-1].content

: 

In [ ]:
from langchain_core.runnables import RunnablePassthrough

simple_summary_prompt = ChatPromptTemplate.from_template("""Please summarize the following piece of text.
Respond in a manner that a 5 year old would understand. Keep it funky!

Text: {input}""")
simple_summary_chain = {"input": RunnablePassthrough()} | simple_summary_prompt | llm

summary_tool = Tool(
    name="funky_summary_tool",
    func=simple_summary_chain.invoke,
    description="Use this tool to do a funky summary. Make sure you get the text to do a summary of first."
)
tools.append(summary_tool)

agent = create_react_agent(
    tools=tools, model=llm, prompt=agent_prompt, debug=True,
)
result = agent.invoke({"messages": "Funky summarize the christmas speech 2023 of Frank-Walter Steinmeier?"})
result["messages"][-1].content

: 

In [ ]:
retrieval_agent_prompt = SystemMessage(content="""
You are a great AI-Assistant to answer question based provided documents. 
You are given access to an retrieval too. Use it to get information you need - do not answer based on general knowledge!
If you do not get the necessary information from the retrieval tool, try to adjust your question.
Always try to search for single facts and not for complete answers!
Only answer based on sources provided and cite the source after each fact!
""")

tool_retrieval_chain = {"input": RunnablePassthrough()} | retrieval_chain

retrieval_tool = Tool(
    name="retrieval_tool",
    func=tool_retrieval_chain.invoke,
    description="Use this tool to retrieve information from a set of documents. Only search for one word facts!",
)

retrieval_agent = create_react_agent(
    tools=[retrieval_tool], model=llm, prompt=retrieval_agent_prompt,
)

result = retrieval_agent.invoke({"messages": "What is the common ground of pub quizzes and llms?"})
print('\n'.join([f"{message.__class__.__name__}: {str(message)}" for message in result["messages"]]))
print(result["messages"][-1].content)

: 

# Memory

In [ ]:

from langgraph.checkpoint.memory import MemorySaver

config = {"configurable": {"thread_id": "1"}}

memory = MemorySaver()
memory_agent = create_react_agent(llm, prompt=retrieval_agent_prompt, tools=[retrieval_tool], checkpointer=memory)

result = memory_agent.invoke({"messages": "What is the common ground of pub quizzes and llms?"}, config)
print('\n'.join([f"{message.__class__.__name__}: {str(message)}" for message in result["messages"]]))

print("--------")

result = memory_agent.invoke({"messages": "Translate that to german."}, config)
print('\n'.join([f"{message.__class__.__name__}: {str(message)}" for message in result["messages"]]))


: 

## Images

In [12]:
import base64
from mimetypes import guess_type
from langchain.prompts import HumanMessagePromptTemplate, ChatPromptTemplate

# Function to encode a local image into data URL 
def local_image_to_data_url(image_path):
    mime_type, _ = guess_type(image_path)
    # Default to png
    if mime_type is None:
        mime_type = 'image/png'

    # Read and encode the image file
    with open(image_path, "rb") as image_file:
        base64_encoded_data = base64.b64encode(image_file.read()).decode('utf-8')

    # Construct the data URL
    return f"data:{mime_type};base64,{base64_encoded_data}"


prompt_template =  HumanMessagePromptTemplate.from_template(
            template=[
                {"type": "text", "text": "Summarize this image"},
                {
                    "type": "image_url",
                    "image_url": "{encoded_image_url}",
                },
            ]
        )

summarize_image_prompt = ChatPromptTemplate.from_messages([prompt_template])

gpt4_image_chain = summarize_image_prompt | llm 

img_file = "PubImages/UserGuide.jpg"
page3_encoded = local_image_to_data_url(img_file)

answer = gpt4_image_chain.invoke(input={"encoded_image_url":page3_encoded})
print(answer.content)

This image showcases a high-tech, futuristic smart home system titled "Tech Invoaters Smart Home System." It depicts a well-organized, digitalized living space where various smart devices and home automation technologies are seamlessly integrated. The setting features a person interacting with multiple screens and devices, surrounded by digital interfaces that control various aspects of the home environment. The image highlights an advanced, interconnected system designed to enhance convenience and efficiency within a modern smart home layout. Additionally, the detailed digital displays and futuristic design elements emphasize the cutting-edge nature of the technology.


In [ ]:
img_file = "data/images/teambudget.jpg"
page3_encoded = local_image_to_data_url(img_file)

answer = gpt4_image_chain.invoke(input={"encoded_image_url":page3_encoded})
print(answer.content)

: 

## Audio

In [ ]:
import openai

client = openai.AzureOpenAI(
    api_key=azure_key,
    azure_endpoint=azure_endpoint,
    azure_deployment=azure_whisper,
    api_version=azure_version,
)

def get_transcript(audio_file):
    if not os.path.exists(audio_file):
        audio_file = "./data/audio/" + audio_file
    client.audio.with_raw_response
    return client.audio.transcriptions.create(
        file=open(audio_file, "rb"),            
        model="whisper",
        language="de",
    ).text

audio_test_file = "./data/audio/newyear2023.mp3"
print(get_transcript(audio_test_file))


: 

In [ ]:
# audio
def get_audio_doc(input):
    return {
        "context": [Document(
            page_content=get_transcript(input),
            metadata={
                "source": "audio"
            }
        )]
    }
audio_summary_chain = get_audio_doc | summary_chain
#answer = audio_summary_chain.invoke("./data/audio/newyear2023.mp3")
#print(answer)


: 

In [ ]:
def available_audio_files(input):
    return "./data/audio/newyear2023.mp3 - new years eve speech 2023 of Olaf Scholz\n"\
        "./data/audio/newyear2016.mp3 - new years eve speech 2016 of Angela Merkel\n" \
        "./data/audio/christmas2019.mp3 - christmas speech 2019 of Frank-Walter Steinmeyer\n"

audio_file_tool = Tool(
    name="audio_file_tool",
    func=available_audio_files,
    description="Use this tool to see which new years eve speeches are available as an audio file."
)

audio_tool = Tool(
    name="audio_transcript_tool",
    func=get_audio_doc,
    description="Use this tool to get the transcript of the speech in a given audio file."
)

agent = create_react_agent(
    tools=[audio_file_tool, audio_tool, summary_tool],
    model=llm,
    prompt=agent_prompt,
    debug=True,
)
agent.invoke({"messages": "Summarize the new years eve speech 2023 of Olaf Scholz using the audio and summary tool?"})


: 

# SQL

In [13]:
SQL_AGENT_PROMPT = """You are an agent designed to interact with a SQL database, to show database outputs as tables and to generate plots.
Given an input question, create a syntactically correct SQL query to run, then look at the results of the query and return the answer.
If no specific number of results is requested, return the top 5 results. If a specific range (e.g. dates) are requested, don't limit the number of results.
You can order the results by a relevant column to return the most interesting examples in the database.
Query for the relevant columns given e question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

Always give the results received from the database. They must be displayed with a custom markdown element (table).
This is an example of how to use it:

```db_result
| Country | TotalCost |
|---------|-----------|
| Germany | 101       |
| Italy   | 104       |
| France  | 98        |
```

Use **bold** markdown syntax to highlight the most relevant part of your answer (e.g. a calculation result).

Before making any query, check the available table names and correct schema of the table to avoid errors.

Hint: to get a specfic part of a datetime, use the strftime(format, date) function with %Y for years, %m for months and %d for days.

Remember: ALWAYS output the returned data! ALWAYS use the custom markdown syntax for this (triple backtick and db_result)!
ALWAYS query the database if any data is requested, even if the user does not refer to the database. Assume that any request refers to DB data and execute the corresponding function.
The database is the single source of truth for your answers. If the data cannot be found in the database, state this truthfully.
y
Double check if you actually used a function to query the DB before including data in your response.

If you need to do any calculations, ALWAYS use the SQL database for this. NEVER use your own knowledge to provide the answer to a mathematical calculation.
For example, to calculate the sum of 5 and 3, run this query: `SELECT 5 + 3 AS result;`

"""

In [14]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///chinook.db")



In [17]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase
from langgraph.prebuilt import create_react_agent

db = SQLDatabase.from_uri("sqlite:///PubSql/chinook.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()

sql_agent = create_react_agent(
    tools=tools,
    model=llm,
    prompt=SQL_AGENT_PROMPT,
)

result = sql_agent.invoke({"messages": "What are the top 5 tracks by length?"})
print('\n'.join([f"{message.__class__.__name__}: {str(message)}" for message in result["messages"]]))
#print('\n'.join([message.content for message in result["messages"]]))

HumanMessage: content='What are the top 5 tracks by length?' additional_kwargs={} response_metadata={} id='db1b70bc-2444-467c-a8b8-baa6ad8a098e'
AIMessage: content='' additional_kwargs={'tool_calls': [{'id': 'call_crNV8PCDkqEOG1MAtoRBdO2s', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 844, 'total_tokens': 857, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_b705f0c291', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'sa

In [ ]:
db.run("SELECT * FROM track LIMIT 5;")

: 

In [ ]:
from pydantic import BaseModel

class WikiJson(BaseModel):
    title: str
    content: str
    summary: str

llm_structured = AzureChatOpenAI(
    api_key=azure_key,
    api_version="2024-12-01-preview",
    azure_deployment="gpt-4o-mini",
    model=azure_deployment,
    azure_endpoint=azure_endpoint,
).with_structured_output(WikiJson)

answer = llm_structured.invoke("What do you know about Pub Quizzes?", seed=1000)
print(answer.model_dump_json(indent=2))

: 